# Vorlesung 5.2
# Speicherklassen

---
 **Hinweis:**
 Diese interaktiven Webseiten beschreiben parallel zu den Vorlesungsfolien das jeweilige Stoffgebiet. Zellen mit C Quelltext können mittels der Tastenkombination \[Shift\] + \[Enter\] kompiliert und ausgeführt werden. Es wird Ihnen empfohlen, Änderungen in diversen Zellen vorzunehmen um ein Gefühl für die Sprache C zu entwickeln.
 
---

## Lokale Variablen

Bisher wurden in Beispielen nur lokale Variablen verwendet. Das bedeutet, dass diese Variablen nur in der Funktion verwendet werden können, in der sie definiert wurden.

In [ ]:
#include <stdio.h>

/* a und faktor sind nur innerhalb dieser Funktion sichtbar, da sie lokal sind.
    Jedes Mal wenn die Funktion aufgerufen wird, werden sie neu initialisiert */
long Faktorielle(long a) {
    long faktor = 1;
    
    /* Wenn a < 0 ist soll eine Warnung ausgegeben werden */
    if(a < 0) {
        printf("a muss positiv sein!\n");
        /* Es wird der Fehlerwert -1 zurückgegeben, der sonst nie auftreten kann */
        return -1;
    }
    
    for(; a > 1; a = a - 1) {
        faktor = faktor * a;
    }
    return faktor;
}

int main() {
    printf("%ld\n", Faktorielle(6));
    
    return 0;
}

### Ein Block in C

Variablen müssen immer am Anfang eines Blockes definiert werden. Ein Block ist ein Programmbereich, der in geschwungenen Klammern `{}` eingefasst ist. Das sind zum Beispiel:   
* Schleifenrümpfe
* Funktionsrümpfe
* Alle anderen Verwendungen von `{}`

Dadurch, dass die Variable `faktor` am Anfang der Funktion definiert wird, nimmt sie bei jedem Aufruf der Funktion Platz im Speicher ein. Wenn allerdings der übergebene Parameter `a` negativ ist, wird die Variable `faktor` nie verwendet. Um diese Verschwendung von Speicherplatz zu vermeiden, kann ein neuer Block geschrieben werden, in dem `faktor` definiert wird.

In [ ]:
/* Faktorielle mit zusätzlichem Block */
#include <stdio.h>

long Faktorielle(long a) {    
    /* Wenn a < 0 ist soll eine Warnung ausgegeben werden */
    if(a < 0) {
        printf("a muss positiv sein!\n");
        /* Es wird der Fehlerwert -1 zurückgegeben, der sonst nie auftreten kann */
        return -1;
    }
    
    /* Jetzt wird faktor noch immer zu Beginn eines Blockes definiert,
        aber erst wenn sichergestellt ist, dass a positiv ist */
    {
        long faktor = 1;
        for(; a > 1; a = a - 1) {
            faktor = faktor * a;
        }
        return faktor;
    }
}

int main() {
    printf("%ld\n", Faktorielle(6));
    
    return 0;
}

In [ ]:
/* Blöcke übernehmen Variablen von Blöcken in denen sie stehen */
long a = 5;
{
    /* a kann hier verwendet werden */
    long b = a + 12;
    a -= b;
    printf("%ld\n", b);
}
/* a wurde verändert */
printf("%ld\n", a);

In [ ]:
/* VORSICHT: Wenn zwei Variablen gleichen Namens in verschiedenen Blöcken existieren, zählt der innerste Block */
long a = 5;
{
    long a;
    a = 1;
    printf("%ld\n", a);
}
printf("%ld\n", a);

### Speicherklassen von lokalen Variablen

Es gibt drei Speicherklassen von lokalen Variablen:
* `auto`
* `register`
* `static`

### auto

Alle Variablen, die nicht mit `register` oder `static` definiert sind, sind automatische Variablen. Das heißt, sie werden am Anfang des Blockes, in dem sie definiert sind, automatisch angelegt und am Endes des Blockes wieder gelöscht. Das Schlüsselwort `auto` muss dabei nicht extra angegeben werden, da ohnehin alle Variablen `auto` sind, wenn nicht explizit anders angegeben.

### register

Variablen, die häufig verwendet werden, können als Speicherklasse `register` definiert werden. Dadurch wird die Variable in einem Register des Prozessors angelegt und nicht im Hauptspeicher, was die Zugriffszeiten verringert. Ob die Variable allerdings wirklich in einem Register angelegt wird, oder doch im Hauptspeicher, ist Abhängig vom Compiler. Mittels `register` können allerdings nur `char`, `long`, `float`, `double` und Zeiger gespeichert werden. Zudem kann die Speicheradresse NICHT mit `&` ermittelt werden, da Register keine Speicheradressen haben.

Diese Variablen werden genau so wie `auto` Variablen am Anfang eines Blockes angelegt und an dessen Ende gelöscht.

**Bemerkung**: Moderne Compiler haben mächtige Optimierungswerkzeuge, die sehr häufig verwendete Variablen automatisch in Registern erstellen/belassen, etc. Zudem muss ein Compiler `register` nicht respektieren und kann die Variable einfach im Hauptspeicher anlegen. Daher ist die Angabe von `register` in modernen Codes nicht sehr geläufig. Es kann aber verwendet werden um klarzustellen, dass schneller Zugriff auf eine bestimmte Variable wichtig ist.

In [ ]:
#include <stdio.h>

/* Alle Variablen werden mit register angelegt um zu zeigen,
    dass schneller Zugriff wichtig ist weil sie oft verwendet werden. */
long Fibonacci(register long maximum) {
    register long fibonacci = 1;
    register long old_term = 0;
    register long temp = 0;
    
    long i = 0;
    for(; i < maximum; ++i) {
        /* printf("%ld\n", fibonacci); */
        temp = fibonacci;
        fibonacci += old_term;
        old_term = temp;
    }
    
    return fibonacci;
}

int main() {
    long term = 90;
    
    printf("Fibonacci Term %ld: %ld\n", term, Fibonacci(term));
    
    return 0;
}

### static

Lokale statische Variablen werden nicht, wie in anderen Speicherklassen, erst am Anfang eines Blockes im Speicher angelegt, sondern schon zu Beginn des gesamten Programmes. Dadurch werden sie auch erst gelöscht, wenn das Programm beendet. Sie können allerdings nur in dem Block verwendet werden, in dem sie definiert wurden. Variablendefinitionen des Typs `static` existieren daher solange das Programm läuft, sind allerdings nur lokal sichtbar, z.B. beim Einsprung in eine Funktion. Wenn eine `static` Variable bei ihrer Definition mit `=` initialisiert wird, passiert das nur beim ersten Durchlauf und wird danach ignoriert.

In [ ]:
/* Beispiel für eine statische Variable als Zähler */
#include <stdio.h>

long AufrufZaehler() {
    /* Eine statische Variable behält ihren Wert,
        da sie so lange existiert bis das gesamte Programm beendet. */
    static long aufrufe = 0;
    return ++aufrufe;
}

int main() {
    printf("%ld\n", AufrufZaehler());
    printf("%ld\n", AufrufZaehler());
    printf("%ld\n", AufrufZaehler());
    printf("%ld\n", AufrufZaehler());

    return 0;
}

## Globale Variablen

Globale Variablen werden außerhalb jedes Blockes definiert. Dadurch sind sie so lange verfügbar bis das Programm beendet, ähnlich wie lokale `static` Variablen. Allerdings können Variablen von JEDEM Block im Programm verändert werden, da sie ja im höchsten "Block", nämlich ganz außerhalb, definiert sind. Da dies zu Problemen mit der Struktur eines Programmes führen kann, sollten globale Variablen vermieden werden soweit es möglich ist.

Globale Variablen werden zwar immer mit `0` initialisiert, sollten aber trotzdem immer per Hand initialisiert werden, um Verwirrungen zu vermeiden.

In [ ]:
#include <stdio.h>
/* Globale Variable, die im gesamten Programm verfügbar ist.
    Dies sollte unbedingt vermieden werden!!! */
const double PI = 3.1415926;

double rad2grad(double rad) {
    /* Die Variable kann hier verwendet werden */
    double grad = rad * 180. / PI;
    
    return grad;
}

int main() {
    /* Aber sie kann auch hier verwendet werden */
    double rad = PI;
    double grad = rad2grad(rad);
    
    printf("%f rad = %f grad\n", rad, grad);
    
    return 0;
}

### Globale Variablen der Speicherklasse `extern`

Wenn sonst keine Speicherklasse angegeben ist, sind globale Variablen `extern`. Das heißt sie können im gesamten Programm und auch in anderen Modulen (anderen .c Dateien) verwendet werden. Um eine Variable in anderen Modulen verwenden zu können, muss sie dort deklariert werden, damit der Compiler weiß, dass in einem anderen Modul eine Variable existiert, die erst später beim Linken verfügbar sein wird (siehe Vorlesung 2.1). Die Variable `PI` im obigen Beispiel ist eine externe globale Variable. In einer anderen .c Datei könnte sie verwendet werden, nachdem sie folgendermaßen deklariert ist:
```C
extern const double PI;
```
Dadurch hat der Compiler alle Informationen über die Variable und weiß durch `extern`, dass diese Variable von einem anderen Modul bereitgestellt wird.

### Globale Variablen der Speicherklasse `static`

Um eine globale Variable zu erstellen, die nur in einem Modul sichtbar ist (also nur in der .c Datei in der sie definiert wurde), wird das Schlüsselwort `static` vorangestellt.

**Anmerkung**: Der Unterschied zwischen globalen Variablen der Typen `static` und `extern` kann hier am Server nicht gezeigt werden, da immer nur eine .c Datei verwendet wird. Testen Sie den Unterschied auf Ihrem Rechner!

In [ ]:
#include <stdio.h>
/* Globale Variable, die nur in dieser .c Datei verfügbar ist */
static const double PI = 3.1415926;

double rad2grad(double rad) {
    double grad = rad * 180. / PI;
    
    return grad;
}

int main() {
    double rad = PI;
    double grad = rad2grad(rad);
    
    printf("%f rad = %f grad\n", rad, grad);
    
    return 0;
}

**WICHTIG**: Variablen sollten **IMMER** so definiert werden, dass der Zugriff auf sie so eingeschränkt wie möglich ist. Wenn man eine Variable erstellt, sollte ihre Definition in dieser Reihenfolge abgewogen werden:

* Lokale Variable
* Wenn die Variable in einer Funktion benötigt wird: Übergabeparameter der Funktion
* Wenn die Variable über mehrere Aufrufe benötigt wird: `static`
* Wenn die Variable an vielen Stellen im Programm benötigt wird: globale Variable (dies sollte vermieden werden)
    * `static`
    * Wenn die Variable in anderen Modulen gelesen werden soll: Eine externe Funktion, die den Wert zurückgibt
    * Wenn die Variable in anderen Modulen geschrieben werden muss: Eine externe Funktion, die den Wert setzt
    * Wenn die Variable wirklich, unbedingt, um jeden Preis in anderen Modulen verwendet werden MUSS, dann als externe Variable
    
Globale Variablen (besonders externe globale Variablen) sollten wirklich nur der allerletzte Ausweg sein und von ihrer Verwendung wird in jedem reputablen Programmierstil abgeraten.